# Artificial and Computational Intelligence Assignment 1

## Problem solving by Uninformed & Informed Search

List only the BITS (Name) of active contributors in this assignment:
1. Kulkarni Sudhanva Prasad (2022AC05006)
2. Pushkar Kumar Verma (2022AC05272)
3. Priyesh Joshi (2022AC05382)
4. Irfan Jahangeer Pathan (2022AC05691)

Things to follow
1.	Use appropriate data structures to represent the graph and the path using python libraries
2.	Provide proper documentation
3.	Find the path and print it

Coding begins here

### 1.	Define the environment in the following block

List the PEAS decription of the problem here in this markdown block

Design the agent as PSA Agent(Problem Solving Agent)
Clear Initial data structures to define the graph and variable declarations is expected
IMPORTATANT: Write distinct code block as below

In [4]:
import copy
import math

In [5]:
#Code Block : Set Initial State (Must handle dynamic inputs)
def get_initial_state(tent_matrix, food_availability):
  """
  Returns initial state
  """
  # Initial state: Requirement of each tent
  initial_state = {}
  for i, ele in enumerate(tent_matrix):
    initial_state[f'tent{i+1}'] = {'adults': ele[0], 'children': ele[1]}

  # Food Availability: Quantity of each food item available
  initial_food_state = {
                            'dhal': food_availability[0],
                            'milk': food_availability[1],
                            'flour': food_availability[2],
                            'rice': food_availability[3],
                            'bread': food_availability[4],
                       }
  return initial_state, initial_food_state

In [6]:
#Code Block : Set the matrix for transition & cost (as relevant for the given problem)
tent_requirements_matrix = [
    [3,6,9,9,2],
    [12,0,36,36,0],
    [2,9,6,6,3],
    [4,0,12,12,0],
    [3,6,9,9,2],
    [2,3,6,6,1],
    [2,9,6,6,3]
]

cost_matrix = [
    [0,3,4,6,9,7,8],
    [3,0,1,3,6,8,9],
    [6,3,2,0,3,5,6],
    [9,6,5,3,0,2,3],
    [6,8,7,5,2,0,1],
    [6,7,6,5,2,0,1],
    [8,9,8,6,3,1,0]
]

In [7]:
#Code Block : Write function to design the Transition Model/Successor function. Ideally this would be called while search algorithms are implemented
def successor(state, food_state):
    current_state = copy.deepcopy(state)
    current_food_state = copy.deepcopy(food_state)
    new_states = []

    # Sort tents by remaining need in descending order
    sorted_tents = sorted(current_state.items(), key=lambda x: x[1]['adults'] + x[1]['children'], reverse=True)

    return sorted_tents

def convert_needed_quantity_in_packets(amount, food_name):

  if food_name in ['milk', 'bread', 'dhal']:
    return int(amount)
  elif food_name == 'rice':
    return math.ceil(amount / 2.5)
  else:
    return math.ceil(amount / 3)

def calculate_people_on_quantity(food_name, no_of_packets):
 
  if food_name in ['dhal', 'bread']:
    return int(no_of_packets)
  elif food_name in ['flour', 'milk']:
    return math.ceil(no_of_packets / 3)
  else:
    return math.ceil((no_of_packets * 2.5) / 3)

In [8]:
#Code block : Write fucntion to handle goal test (Must handle dynamic inputs). Ideally this would be called while search algorithms are implemented
def is_goal_state(state, food_state):
    return all(food == 0 for food in food_state.values())

def heuristic(no_of_adults, no_of_children):
    # commute requirements adults
    req_adults_weight = (no_of_adults) + (no_of_adults * 3) + (no_of_adults * 3)
    req_adults_commutes = math.ceil(req_adults_weight / 15)

    # commute requirements children milk
    req_children_milk = no_of_children * 3 
    req_children_milk_commutes = math.ceil(req_children_milk / 15)

    # commute requirements children bread
    req_children_bread = no_of_children
    req_bread_commutes = math.ceil(req_children_bread / 15)

    # total_commutes 
    total_commutes = req_adults_commutes + req_children_milk_commutes + req_bread_commutes
    return total_commutes

### 2.	Local Beam Search

In [9]:
#Code Block : Function for algorithm 1 implementation
from queue import PriorityQueue

def local_beam_search(initial_state, initial_food_state, beam_width):
    # Taking initial state
    current_state = initial_state
    current_food_state = initial_food_state

    # Data structure to store explored states
    explored_states = list()

    # Commits track
    total_commuts = 0

    # food distribution track
    distributed_food = {}

    successors = successor(current_state, current_food_state)
    successors.sort(key=lambda x: heuristic(*x[1].values()), reverse=False)

    n=len(successors)
    idx=0

    q=PriorityQueue(maxsize=beam_width)
    for i in range(beam_width):
        q.put((i,successors.pop(0)))
        n-=1
        idx+=1
    
    while not q.empty():
        # Checking if current state is a goal state
        if is_goal_state(current_state, current_food_state):
            print("path has been found.")
            return explored_states, total_commuts, distributed_food
        
        # Fetching a successor with least heuristic value
        item = q.get()
        next_state, next_requirement = item[1]
        print(f"Chosen state for exploration: {next_state}")

        # Extracting number of adults and children
        no_of_adults, no_of_children = next_requirement['adults'], next_requirement['children']

        # Calculating requirement in packets
        dhal_packets_required = no_of_adults
        flour_packets_required = no_of_adults * 3 
        rice_packets_required = math.ceil((no_of_adults * 2.5) / 3)
        milk_packets_required = no_of_children * 3
        bread_packets_required = no_of_children

        # Updating the original food state
        new_food_state = copy.deepcopy(current_food_state)
        new_food_state['dhal'] = max((new_food_state['dhal'] - dhal_packets_required), 0)
        new_food_state['milk'] = max((new_food_state['milk'] - milk_packets_required), 0)
        new_food_state['flour'] = max((new_food_state['flour'] - flour_packets_required), 0)
        new_food_state['rice'] = max((new_food_state['rice'] - rice_packets_required), 0)
        new_food_state['bread'] = max((new_food_state['bread'] - bread_packets_required), 0)
        
        # Tracking distributed food
        distributed_food[next_state] = {
            "dhal": current_food_state['dhal'] - new_food_state['dhal'],
            "milk": current_food_state['milk'] - new_food_state['milk'],
            "flour": current_food_state['flour'] - new_food_state['flour'],
            "rice": current_food_state['rice'] - new_food_state['rice'],
            "bread": current_food_state['bread'] - new_food_state['bread']
        }
        print(f"Distributed food in {next_state}: {distributed_food[next_state]}")

        if n:
            q.put((idx, successors.pop(0)))
            idx+=1
            n-=1
        explored_states.append(str(next_state))
        total_commuts += heuristic(no_of_adults, no_of_children)
    
    return explored_states, total_commuts, distributed_food

### 3.	Greedy Best First Search

In [10]:
#Code Block : Function for algorithm 2 implementation
def greedy_best_first_search(initial_state, initial_food_state):
    # Taking initial state
    current_state = initial_state
    current_food_state = initial_food_state

    # Data structure to store explored states
    explored_states = list()

    # Commits track
    total_commuts = 0

    # food distribution track
    distributed_food = {}
    while True:
        # Checking if current state is a goal state
        if is_goal_state(current_state, current_food_state):
            print("path has been found.")
            return explored_states, total_commuts, distributed_food
        
        # Calculating heuristic value for successors
        successors = successor(current_state, current_food_state)
        successors.sort(key=lambda x: heuristic(*x[1].values()), reverse=False)

        # Fetching a successor with least heuristic value
        next_state, next_requirement = successors.pop(0)
        print(f"Chosen state for exploration: {next_state}")

        # Extracting number of adults and children
        no_of_adults, no_of_children = next_requirement['adults'], next_requirement['children']

        # Calculating requirement in packets
        dhal_packets_required = no_of_adults
        flour_packets_required = no_of_adults * 3 
        rice_packets_required = math.ceil((no_of_adults * 2.5) / 3)
        milk_packets_required = no_of_children * 3
        bread_packets_required = no_of_children

        # Updating the original food state
        new_food_state = copy.deepcopy(current_food_state)
        new_food_state['dhal'] = max((new_food_state['dhal'] - dhal_packets_required), 0)
        new_food_state['milk'] = max((new_food_state['milk'] - milk_packets_required), 0)
        new_food_state['flour'] = max((new_food_state['flour'] - flour_packets_required), 0)
        new_food_state['rice'] = max((new_food_state['rice'] - rice_packets_required), 0)
        new_food_state['bread'] = max((new_food_state['bread'] - bread_packets_required), 0)
        
        # Tracking distributed food
        distributed_food[next_state] = {
            "dhal": current_food_state['dhal'] - new_food_state['dhal'],
            "milk": current_food_state['milk'] - new_food_state['milk'],
            "flour": current_food_state['flour'] - new_food_state['flour'],
            "rice": current_food_state['rice'] - new_food_state['rice'],
            "bread": current_food_state['bread'] - new_food_state['bread']
        }
        print(f"Distributed food in {next_state}: {distributed_food[next_state]}")

        # Removing chosen state from state transition as it is closed now
        new_state = {k:v for k,v in current_state.items() if k != next_state}
        current_state = new_state
        current_food_state = new_food_state
        explored_states.append(str(next_state))
        total_commuts += heuristic(no_of_adults, no_of_children)

### DYNAMIC INPUT

IMPORTANT : Dynamic Input must be got in this section. Display the possible states to choose from:
This is applicable for all the relevent problems as mentioned in the question.

In [11]:
#Code Block : Function & call to get inputs (start/end state)
# Defining tent matrix representing input config
tent_matrix = [
    [3,2],
    [12,0],
    [2,3],
    [4,0],
    [3,2],
    [2,1],
    [2,3]
]

# Defining total food availability
food_availability = [15, 28, 12, 20, 10]

beam_width = 2

def run_algorithm_return_states(tent_matrix, food_availability, algo_type):
    # Fetching initial state based on above two data structures
    initial_state, initial_food_state = get_initial_state(tent_matrix, food_availability)

    # Invocation of the algorithm
    if algo_type == "greedy_best_first_search":
        path, total_commutes, distribution_of_food = greedy_best_first_search(initial_state, initial_food_state)
    else:
        path, total_commutes, distribution_of_food = local_beam_search(initial_state, initial_food_state, beam_width)

    print(f"Optimal path using greedy best first search: {' --> '.join(path)}")
    print(f"Total commutes required for the bot to distribute food: {total_commutes}")

### 4.	Calling the search algorithms
(For bidirectional search in below sections first part can be used as per Hint provided. Under second section other combinations as per Hint or your choice of 2 algorithms can be called .As an analyst suggest suitable approximation in the comparitive analysis section)

In [16]:
#Invoke algorithm 1 (Should Print the solution, path, cost etc., (As mentioned in the problem))
run_algorithm_return_states(tent_matrix, food_availability, "local_beam_search")

Chosen state for exploration: tent4
Distributed food in tent4: {'dhal': 4, 'milk': 0, 'flour': 12, 'rice': 4, 'bread': 0}
Chosen state for exploration: tent3
Distributed food in tent3: {'dhal': 2, 'milk': 9, 'flour': 6, 'rice': 2, 'bread': 3}
Chosen state for exploration: tent7
Distributed food in tent7: {'dhal': 2, 'milk': 9, 'flour': 6, 'rice': 2, 'bread': 3}
Chosen state for exploration: tent6
Distributed food in tent6: {'dhal': 2, 'milk': 3, 'flour': 6, 'rice': 2, 'bread': 1}
Chosen state for exploration: tent1
Distributed food in tent1: {'dhal': 3, 'milk': 6, 'flour': 9, 'rice': 3, 'bread': 2}
Chosen state for exploration: tent5
Distributed food in tent5: {'dhal': 3, 'milk': 6, 'flour': 9, 'rice': 3, 'bread': 2}
Chosen state for exploration: tent2
Distributed food in tent2: {'dhal': 12, 'milk': 0, 'flour': 12, 'rice': 10, 'bread': 0}
Optimal path using greedy best first search: tent4 --> tent3 --> tent7 --> tent6 --> tent1 --> tent5 --> tent2
Total commutes required for the bot to

In [13]:
#Invoke algorithm 2 (Should Print the solution, path, cost etc., (As mentioned in the problem))
run_algorithm_return_states(tent_matrix, food_availability, "greedy_best_first_search")

Chosen state for exploration: tent4
Distributed food in tent4: {'dhal': 4, 'milk': 0, 'flour': 12, 'rice': 4, 'bread': 0}
Chosen state for exploration: tent3
Distributed food in tent3: {'dhal': 2, 'milk': 9, 'flour': 0, 'rice': 2, 'bread': 3}
Chosen state for exploration: tent7
Distributed food in tent7: {'dhal': 2, 'milk': 9, 'flour': 0, 'rice': 2, 'bread': 3}
Chosen state for exploration: tent6
Distributed food in tent6: {'dhal': 2, 'milk': 3, 'flour': 0, 'rice': 2, 'bread': 1}
Chosen state for exploration: tent1
Distributed food in tent1: {'dhal': 3, 'milk': 6, 'flour': 0, 'rice': 3, 'bread': 2}
Chosen state for exploration: tent5
Distributed food in tent5: {'dhal': 2, 'milk': 1, 'flour': 0, 'rice': 3, 'bread': 1}
Chosen state for exploration: tent2
Distributed food in tent2: {'dhal': 0, 'milk': 0, 'flour': 0, 'rice': 4, 'bread': 0}
path has been found.
Optimal path using greedy best first search: tent4 --> tent3 --> tent7 --> tent6 --> tent1 --> tent5 --> tent2
Total commutes requi

### 5.	Comparitive Analysis

In [14]:
#Code Block : Print the Time & Space complexity of algorithm 1
print("Local Beam Search Algorithm Analysis:")
print("Time Complexity: O(I * (N * M log M + K))")
print("Space Complexity: O(K * (M + N) + Tracking Distributed Food)")
print("where, K is the beam width, N is number of tents, M is number of food items and I is number of iterations")

Local Beam Search Algorithm Analysis:
Time Complexity: O(I * (N * M log M + K))
Space Complexity: O(K * (M + N) + Tracking Distributed Food)
where, K is the beam width, N is number of tents, M is number of food items and I is number of iterations


In [15]:
#Code Block : Print the Time & Space complexity of algorithm 2
print("Greedy Best First Search Algorithm Analysis:")
print("Time Complexity: O(N * (N log N) * 2^N)")
print("Space Complexity: O(N * (M + K) + N)")
print("where, N is the number of tents, M is the number of adults, and K is the number of children.")

Greedy Best First Search Algorithm Analysis:
Time Complexity: O(N * (N log N) * 2^N)
Space Complexity: O(N * (M + K) + N)
where, N is the number of tents, M is the number of adults, and K is the number of children.


### 6.	Provide your comparitive analysis or findings in no more than 3 lines in below section

Comparison : 
* Greedy Best-First Search explores the search space based on a <b>single best</b> heuristic value, while Local Beam Search explores <b>multiple states</b> in parallel with the hope of finding better solutions.

* Greedy Best-First Search can get stuck in local optima, while Local Beam Search has a better chance of escaping local optima with multiple beams.

* Greedy Best-First Search may be more memory-efficient than Local Beam Search for <b>small</b> problem instances, but Local Beam Search can handle <b>larger</b> problem instances more effectively.

* Neither algorithm guarantees an optimal solution, and the performance of both can vary depending on the problem's characteristics and the choice of heuristics.
